In [16]:
from time import time

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import time
import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST as FMNIST
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms
import tensorflow as tf
import random as r
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook
import copy

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC

import matplotlib.pyplot as plt

In [17]:
path="../../Data/"
attacks_types = {
    'normal': 'normal','back': 'dos','buffer_overflow': 'u2r','ftp_write': 'r2l','guess_passwd': 'r2l',
'imap': 'r2l','ipsweep': 'probe','land': 'dos','loadmodule': 'u2r','multihop': 'r2l','neptune': 'dos',
'nmap': 'probe','perl': 'u2r','phf': 'r2l','pod': 'dos','portsweep': 'probe','rootkit': 'u2r','satan': 'probe',
'smurf': 'dos','spy': 'r2l','teardrop': 'dos','warezclient': 'r2l','warezmaster': 'r2l',
}
cols ="""duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,
urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,
num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,
count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,
diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,
dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,
dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate"""
  
columns =[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())
print(len(columns))
columns.append('target')
print(len(columns))

attack_categories=["dos","u2r","r2l",'probe','normal']
df = pd.read_csv(path+"kddcup.data_10_percent.gz", names = columns)
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])
del df['target']
df.head()
num_cols = df._get_numeric_data().columns
  
cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('Attack Type')
def getuniquevalues(columnname):
    values={}
    i=0
    for entry in df[columnname]:
        if entry not in values:
            values[entry]=i
            i+=1
    return values
for col in cate_cols:
    df[col]=df[col].map(getuniquevalues(col))
data=df.to_numpy()
Y=df['Attack Type'].map(getuniquevalues('Attack Type'))
Y=Y.to_numpy()
X=data[:,:-1]
print(Y.shape)
print(X.shape)
print(getuniquevalues('Attack Type'))
def normalized(x,y):
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y, shuffle=True)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()
    return x.numpy(), x_test.numpy(), y.numpy(), y_test.numpy(), scaler
xtrain, x_test, ytrain, y_test,scaler= normalized(X,Y)

41
42
(494021,)
(494021, 41)
{'normal': 0, 'u2r': 1, 'dos': 2, 'r2l': 3, 'probe': 4}


In [18]:
class NeuralHD:
    def __init__(self, classes : int, features : int, dim : int = 400):
        #Configure for hdb, hdc, and hde classes
        self.param=Config.config
        self.param['nClasses'] = classes
        self.param['nFeatures']= features
        #hypervector size
        self.param['D']=dim
        #encoder
        self.hde=None
        #classifier
        self.hdc=None
    def __call__(self, x : torch.Tensor):
        #True iff the model has been trained
        assert self.hde!=None and self.hdc!=None
        #return predicted values
        return self.predict(x)
    def predict(self,x):
        #Get hypervectors for all data points
        trainencoded=self.hde.encodeData(x)
        #return predictions based on similarity to classification hypervectors
        return np.array(self.hdc.predict(trainencoded))
    def fit(self,traindata, trainlabels,
                   epochs,
                   regenloops,  # list of effective dimensions to reach 
                   percentDrop # drop/regen rate 
                    ):
        
        # Initialize basis & classifier
        hdb = HDB.HD_basis(HDB.Generator.Vanilla, self.param)
        # store generated basis
        basis = hdb.getBasis()
        # updata params after basis generation
        self.param = hdb.getParam()
        # make encoder based on basis
        self.hde = HDE.HD_encoder(basis)
        # find encoded training vectors
        trainencoded = self.hde.encodeData(traindata)
        # Initialize classification hypervectors
        self.hdc = HDC.HD_classifier(self.param["D"], self.param["nClasses"], 0)

        # calculate amount of dropped dimensions based on percent and original dimension
        amountDrop = int(percentDrop * self.hdc.D)#self.param.D?
        print("Updating times:", regenloops)

        for i in range(regenloops+1): # For each eDs to reach, will checkpoints
            print("regenloop: " + str(i))
            # train for x epochs
            perfect=self.trainreploop(epochs,trainencoded,trainlabels)
            #if its the last regeneration training, stop before doing another dimension drop; stop if 100% accuracy
            if i==regenloops or perfect:
                return #self.hdc,self.hde - unnecessary now that hdc and hde are within a class
            print("regeneration")
            #do the dimension drop and regeneration
            trainencoded=self.regen(hdb,amountDrop,traindata)
        return "error","error"
    
    def trainreploop(self,epochs,trainencoded,trainlabels):
        # Do the train 
        for j in range(epochs):
            # do one pass of training
            train_acc = 100 * self.hdc.fit(trainencoded, trainlabels, self.param)
            #Test accuracy used to be in here but I took it out because we dont usually know it
            print("Train: %.2f \t \t Test: "%(train_acc))
            # If accuracy is 100, finish
            if train_acc == 100:
                return True
        return False
    def regen(self,hdb,amountDrop,traindata):
        #make order of dimensions according to variaince; also store variances <-- unnecessary?
        var, orders = self.hdc.evaluateBasis()
        #drop dimensions with lowest variance
        toDrop = orders[:amountDrop]
        # print("Variances stats: max %.2f, min %.2f, mean %.2f"%(max(var),min(var),np.mean(var)))
        #update basis by randomizing dimension
        hdb.updateBasis(toDrop)
        # move the new basis into the encoder
        self.hde.updateBasis(hdb.basis)
        # normalize previous classes so retrain has enough effect
        self.hdc.updateClasses()
        # get new encoded training data
        return self.hde.encodeData(traindata)    

## Model Setup

In [19]:
model=NeuralHD(5,xtrain.shape[1],30)

## Model Training

In [20]:
model.fit(xtrain,ytrain,2,5,.1)

Updating times: 5
regenloop: 0
Train: 98.24 	 	 Test: 
Train: 98.54 	 	 Test: 
regeneration
Updating basis......
regenloop: 1
Train: 98.41 	 	 Test: 
Train: 98.59 	 	 Test: 
regeneration
Updating basis......
regenloop: 2
Train: 98.46 	 	 Test: 
Train: 98.60 	 	 Test: 
regeneration
Updating basis......
regenloop: 3
Train: 98.48 	 	 Test: 
Train: 98.61 	 	 Test: 
regeneration
Updating basis......
regenloop: 4
Train: 98.47 	 	 Test: 
Train: 98.65 	 	 Test: 
regeneration
Updating basis......
regenloop: 5
Train: 98.46 	 	 Test: 
Train: 98.60 	 	 Test: 


## Model Prediction

In [22]:
for i in range (0,model.param['nClasses']):
    yhat= model(x_test[y_test==i])
    acc = (yhat==i).mean()
    print('class '+str(i)+' accuracy: ' +str(acc))
    print('points: '+ str(len(yhat)))

class 0 accuracy: 0.9954973393368809
points: 24430
class 1 accuracy: 0.0
points: 16
class 2 accuracy: 0.9985575890254007
points: 97753
class 3 accuracy: 0.17346938775510204
points: 294
class 4 accuracy: 0.49358341559723595
points: 1013


In [ ]:
# Generate one random vector of desired length and generation type
def generate_vector(vector_length, vector_type, param):
    if vector_type == "Gaussian":
        mu = param["mu"]
        sigma = param["sigma"]
        return np.random.normal(mu, sigma, vector_length)
    else:
        raise Exception("Vector type %s not recognized. Abort.\n" % vector_type)

def vanilla(param):

    #sys.stderr.write("Generating vanilla HD basis of shape... ")
    basis = []
    #for i in range(param["D"]):
    #for _ in tqdm_notebook(range(self.param["D"]), desc='vectors'):
    for _ in range(param["D"]):
        basis.append(generate_vector(param["nFeatures"], param["vector"], param))
    basis = np.asarray(basis)

# Load basis from a file



        #sys.stderr.write(str(self.basis.shape)+"\n")
def updateBasis(basis, param, toChange = None):
    print("Updating basis......")# at the following indices: (None means changing everything)")
    #print(toChange)
    for i in toChange:
        basis[i] = generate_vector(param["nFeatures"], param["vector"], param)
    return


NameError: name 'param' is not defined